In [1]:
# Getting all market ids from upbit, cell id: 1

import requests
def get_market_data_requests():
    try:
        url = "https://sg-api.upbit.com/v1/market/all"
        
        params = {
            "is_details": "true"
        }
        
        # Headers
        headers = {
            "accept": "application/json"
        }
        
        # Make the request
        response = requests.get(url, params=params, headers=headers)
        
        # Check if request was successful
        response.raise_for_status()
        
        # Parse and return JSON data
        return response.json()
    except Exception as e:
            print(f"Curl error: {e}")
            return None
        

print(get_market_data_requests())

[{'market': 'SGD-ETH', 'english_name': 'Ethereum', 'market_warning': 'NONE'}, {'market': 'SGD-XRP', 'english_name': 'Ripple', 'market_warning': 'NONE'}, {'market': 'SGD-BTC', 'english_name': 'Bitcoin', 'market_warning': 'NONE'}, {'market': 'SGD-USDT', 'english_name': 'Tether', 'market_warning': 'NONE'}, {'market': 'SGD-SOL', 'english_name': 'Solana', 'market_warning': 'NONE'}]


In [2]:
# Obtain data from upbit, cell id: 2

import requests
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def parse_date_in_upbit_format(to):
    """
    format date in upbit required format
     yyyy-MM-dd'T'HH:mm:ssXXX or yyyy-MM-dd HH:mm:ss.
    """
    try:
        try:
            datetime.strptime(to, '%Y-%m-%dT%H:%M:%S%z')
        except ValueError:
            datetime.strptime(to, '%Y-%m-%d %H:%M:%S')
        return to
    except ValueError:
        print("Error: Invalid date format. Use 'yyyy-MM-dd'T'HH:mm:ssXXX' or 'yyyy-MM-dd HH:mm:ss'")
        return None

def get_market_data_upbit(market_id, candle_type, units=None, count=100, to=None):
    """
    Get market data from Upbit
    market_id: str (e.g., 'KRW-BTC')
    candle_type: str ('minutes', 'days', 'weeks', etc.)
    units: str (required for minutes - '1', '3', '5', etc.)
    count: int (number of candles to retrieve)
    to: timestamp
    """
    try:
        valid_candles = ["minutes", "days", "weeks", "months"]
        if candle_type not in valid_candles:
            print(f"Error: candle_type must be one of {valid_candles}")
            return None

        base_url = f"https://api.upbit.com/v1/candles/{candle_type}"
        if candle_type == "minutes":
            if not units:
                print("Error: units parameter required for minutes candle type")
                return None
            base_url += f"/{units}"

        params = {
            "market": market_id,
            "count": count
        }
        if to is not None:
            parsed_date = parse_date_in_upbit_format(to)
            if parsed_date is not None:
                params["to"] = parsed_date
        

        response = requests.get(
            base_url,
            params=params,
            headers={"accept": "application/json"}
        )
        
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None
    

def graph_prices_upbit(data_input, title):
    dates = [datetime.strptime(d['candle_date_time_utc'], '%Y-%m-%dT%H:%M:%S') for d in data_input]
    opening_prices = [d['opening_price'] for d in data_input]
    high_prices = [d['high_price'] for d in data_input]
    low_prices = [d['low_price'] for d in data_input]
    closing_prices = [d['trade_price'] for d in data_input]

    plt.figure(figsize=(12, 6))
    plt.plot(dates, closing_prices, label='Closing Price', color='blue')
    plt.plot(dates, opening_prices, label='Opening Price', color='green', alpha=0.5)
    plt.fill_between(dates, high_prices, low_prices, alpha=0.2, color='gray', label='Price Range')

    plt.title(title + ' Price Chart')
    plt.xlabel('Date')
    plt.ylabel('Price (USDT)')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()

    # Format date axis
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gcf().autofmt_xdate()

    plt.margins(x=0.01)

    # Show the plot
    plt.tight_layout()
    plt.show()

instrument="USDT-BTC"
data_btc_usdt = get_market_data_upbit(
        market_id=instrument,
        candle_type="days",
        count=200
    )
# graph_prices_upbit(data_btc_usdt, instrument)

print(data_btc_usdt)


[{'market': 'USDT-BTC', 'candle_date_time_utc': '2024-11-21T00:00:00', 'candle_date_time_kst': '2024-11-21T09:00:00', 'opening_price': 93549.04, 'high_price': 98000.0, 'low_price': 93549.03, 'trade_price': 97199.99, 'timestamp': 1732176185254, 'candle_acc_trade_price': 551333.26930943, 'candle_acc_trade_volume': 5.73905537, 'prev_closing_price': 93541.96, 'change_price': 3658.03, 'change_rate': 0.039105766}, {'market': 'USDT-BTC', 'candle_date_time_utc': '2024-11-20T00:00:00', 'candle_date_time_kst': '2024-11-20T09:00:00', 'opening_price': 91910.32, 'high_price': 94699.99, 'low_price': 90579.0, 'trade_price': 93541.96, 'timestamp': 1732146958507, 'candle_acc_trade_price': 1202013.32677514, 'candle_acc_trade_volume': 12.98721945, 'prev_closing_price': 91910.03, 'change_price': 1631.93, 'change_rate': 0.0177557335}, {'market': 'USDT-BTC', 'candle_date_time_utc': '2024-11-19T00:00:00', 'candle_date_time_kst': '2024-11-19T09:00:00', 'opening_price': 90130.65, 'high_price': 93891.1, 'low_pr

In [3]:
# Download USDT-BTC data from upbit, cell id: 3
import pandas as pd
from datetime import datetime, timedelta
import os

def download_data_from_upbit(market_id, candle_type, start_date=None, end_date=None, filename="btc_prices.csv"):

    try: 
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        
        if end_date is None:
            end_date = datetime.now()
        if start_date is None:
            start_date = end_date - timedelta(days=200)
            
        if os.path.exists(filename):
            existing_data = pd.read_csv(filename)
            existing_data['candle_date_time_utc'] = pd.to_datetime(existing_data['candle_date_time_utc'])
            print(f"Loaded {len(existing_data)} existing records")
        else:
            existing_data = pd.DataFrame()
            print("No existing data found")
        
        # Initialize empty list for new data
        all_new_data = []
        current_date = end_date
        
        while current_date >= start_date:
            print(f"Downloading data for date: {current_date.strftime('%Y-%m-%d')}")
            
            date_str = current_date.strftime('%Y-%m-%d %H:%M:%S')
            
            chunk_data = get_market_data_upbit(
                market_id=market_id,
                candle_type=candle_type,
                count=200,
                to=date_str
            )
            
            if chunk_data is None or len(chunk_data) == 0:
                print("No data received from API")
                break
                
            all_new_data.extend(chunk_data)
            
            oldest_date = datetime.strptime(chunk_data[-1]['candle_date_time_utc'], '%Y-%m-%dT%H:%M:%S')
            current_date = oldest_date - timedelta(days=1)
        
        if all_new_data:
            new_df = pd.DataFrame(all_new_data)
            new_df['date'] = pd.to_datetime(new_df['candle_date_time_utc'])
            new_df['p_n_l'] = new_df['change_rate'] * 100 
            new_df = new_df[['date', 'opening_price', 'trade_price', 'high_price', 'low_price', 'p_n_l']]
            new_df.columns = ['date', 'open', 'close', 'high', 'low', 'p_n_l']
            
            print(f"Downloaded {len(new_df)} new records")
            
            if not existing_data.empty:
                combined_df = pd.concat([existing_data, new_df])
                combined_df = combined_df.drop_duplicates(subset='date', keep='first')
                combined_df = combined_df.sort_values('date', ascending=False)
            else:
                combined_df = new_df
                
            combined_df.to_csv(filename, index=False)
            print(f"Saved {len(combined_df)} total records to {filename}")
            
            return combined_df
        else:
            print("No new data to add")
            return existing_data
    except Exception as e:
        print(f"Unexpected error: {e}")

data = download_data_from_upbit(
    market_id="USDT-BTC",
    candle_type="days",
    start_date=(datetime.now() - timedelta(days=800)).strftime('%Y-%m-%d'),
    end_date=datetime.now().strftime('%Y-%m-%d'),
    filename="./upbit_data/usdt_btc.csv"
)

Unexpected error: 'candle_date_time_utc'


In [4]:
# download USDT-BTC from binance, cell id: 4
import requests
import pandas as pd
from datetime import datetime, timedelta
import os

def get_binance_data(symbol="BTCUSDT", interval="1d", limit=100):
    endpoint = "https://api.binance.com/api/v3/klines"
    
    params = {
        "symbol": symbol,
        "interval": interval,
        "limit": limit
    }
    
    try:
        response = requests.get(endpoint, params=params)
        data = response.json()
        
        df = pd.DataFrame(data, columns=[
            'timestamp', 'open', 'high', 'low', 'close',
            'volume', 'close_time', 'quote_volume', 'trades',
            'taker_buy_base', 'taker_buy_quote', 'ignore'
        ])
        
        df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
        
        for col in ['open', 'high', 'low', 'close', 'volume']:
            df[col] = df[col].astype(float)
        
        df['p_n_l'] = ((df['close'] - df['open']) / df['open']) * 100
        
        df = df[['date', 'open', 'close', 'high', 'low', 'p_n_l']]
        
        df = df.sort_values('date', ascending=False)
        
        return df
        
    except Exception as e:
        print(f"Error: {e}")
        return None

def update_binance_data(filename='./binance_data/usdt_btc.csv', symbol="BTCUSDT", limit=800):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    new_data = get_binance_data(symbol=symbol, limit=limit)
    if new_data is None:
        print("Failed to fetch new data")
        return None
    
    try:
        # Check if file exists and load existing data
        if os.path.exists(filename):
            existing_data = pd.read_csv(filename)
            existing_data['date'] = pd.to_datetime(existing_data['date'])
            print(f"Loaded {len(existing_data)} existing records")
            
            # Combine existing and new data
            combined_data = pd.concat([existing_data, new_data])
            
            # Remove duplicates keeping the newest data
            combined_data = combined_data.drop_duplicates(subset='date', keep='first')
            
            # Sort by date
            combined_data = combined_data.sort_values('date', ascending=False)
            
            print(f"Updated data: {len(combined_data)} records")
        else:
            combined_data = new_data
            print(f"Created new data file with {len(combined_data)} records")
        
        # Save to CSV
        combined_data.to_csv(filename, index=False)
        print(f"Data saved to {filename}")
        
        return combined_data
        
    except Exception as e:
        print(f"Error updating data: {e}")
        return None

# Run the update
updated_data = update_binance_data(
    filename='./binance_data/usdt_btc.csv',
    symbol="BTCUSDT",
    limit=800
)

if updated_data is not None:
    print("\nFirst few rows of updated data:")
    print(updated_data.head())

Loaded 796 existing records
Updated data: 801 records
Data saved to ./binance_data/usdt_btc.csv

First few rows of updated data:
          date      open     close      high      low     p_n_l
799 2024-11-21  94286.56  96948.01  97852.00  94040.0  2.822725
798 2024-11-20  92310.80  94286.56  94831.97  91500.0  2.140335
0   2024-11-19  90464.07  92310.79  93905.51  90357.0  2.041385
1   2024-11-18  89855.98  90464.08  92594.00  89376.9  0.676750
2   2024-11-17  90587.98  89855.99  91449.99  88722.0 -0.808043


In [5]:
# Clean data, make sure dates from both upbit & binance are the same, cell id: 5
import pandas as pd

def sync_csv_files(file1_path, file2_path, output_path1=None, output_path2=None):

    try:
        # Read CSV files
        df1 = pd.read_csv(file1_path)
        df2 = pd.read_csv(file2_path)
        
        df1['date'] = pd.to_datetime(df1['date'])
        df2['date'] = pd.to_datetime(df2['date'])
        
        common_dates = set(df1['date']).intersection(set(df2['date']))
        
        df1_sync = df1[df1['date'].isin(common_dates)].sort_values('date', ascending=False)
        df2_sync = df2[df2['date'].isin(common_dates)].sort_values('date', ascending=False)
        
        print(f"Original rows in file 1: {len(df1)}")
        print(f"Original rows in file 2: {len(df2)}")
        print(f"Rows after synchronization: {len(df1_sync)}")
        
        if output_path1:
            df1_sync.to_csv(output_path1, index=False)
            print(f"Saved synchronized file 1 to: {output_path1}")
        
        if output_path2:
            df2_sync.to_csv(output_path2, index=False)
            print(f"Saved synchronized file 2 to: {output_path2}")
        
        return df1_sync, df2_sync
        
    except Exception as e:
        print(f"Error: {e}")
        return None, None

file1 = "./binance_data/usdt_btc.csv"
file2 = "./upbit_data/usdt_btc.csv"

df1_sync, df2_sync = sync_csv_files(
    file1,
    file2,
    output_path1="./binance_data/usdt_btc.csv",
    output_path2="./upbit_data/usdt_btc.csv"
)
print("\nDone")


Original rows in file 1: 801
Original rows in file 2: 796
Rows after synchronization: 796
Saved synchronized file 1 to: ./binance_data/usdt_btc.csv
Saved synchronized file 2 to: ./upbit_data/usdt_btc.csv

Done
